## **Important Note**

## I began doing the project in this notebook, however I moved to Google Colab for more GPU time, so this notebook is incomplete. Please see the notebook "Final Binary Brain Tumor Classification," for the relevant models or the notebook called "All Models for Binary Brain Tumor Classification" for all model iterations.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Set random state for numpy operations
from numpy.random import seed
seed(2)
# Set random state for tensorflow operations
from tensorflow.random import set_seed
set_seed(3)
# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix
import cv2
import PIL

# **Business Understanding**

Brain tumors in particular are very difficult to diagnose from an MRI image, and artificial intelligence methods of identifying and classifying tumors are oftentimes more accurate than manual identification by a radiologist. That is why the development of neural networks and other AI processes for tumor classification is so valuable and important.

The survival rate for patients diagnosed with a brain tumor is around 35%. This survival rate could be increased if tumors could be identified earlier and more accurately, which AI methods could help with. Additionally, in third world countries, seasoned neurosurgeons (a neurosurgeon is required to make the diagnoses from looking at the MRI) are hard to come by, so a machine learning tool (Decision Support Tool) which could accurately identify tumors would be of great value in these developing nations. This decision support tool would be beneficial to the health industry, and the target audience would be Doctors without Borders, an organization which sends doctors from the US to developing countries to help improve their healthcare.

In [ ]:

def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['acc'])
    ax1.plot(history.history['val_acc'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()

In [ ]:
# Set up ImageDataGenerator
train_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=([0.6,1]),
                                   rotation_range=10,                           
                                   brightness_range=([0.6, 1.5]),
                                   horizontal_flip=True,
                                   validation_split=0.06) # this will set aside a part of training set for validation data
test_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=([0.6,1]),
                                   rotation_range=10,
                                   brightness_range=([0.6,1.5]),
                                   horizontal_flip=True)
# Bring the data in
train_generator = train_imagegen.flow_from_directory(
                                    '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training',
                                    classes={'no_tumor_train':0,
                                            'AllTumorsTrain':1},
                                    target_size=(200,200),
                                    batch_size=20,
                                    seed=42,
                                    class_mode='binary',
                                    subset='training')

test_generator = test_imagegen.flow_from_directory(
                                    '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Testing',
                                     classes={'no_tumor_test':0,
                                            'AllTumorsTest':1},
                                    target_size=(200,200),
                                    batch_size=20,
                                    seed=42,
                                    class_mode='binary')

val_generator = train_imagegen.flow_from_directory(
                                    '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training',
                                     classes={'no_tumor_train':0,
                                            'AllTumorsTrain':1},
                                    target_size=(200,200),
                                    batch_size=20,
                                    seed=42,
                                    class_mode='binary',
                                    subset='validation')

### **Important Note**

One important thing to note is that throughout the modelling process, I use train_generator as my training data images, and testing_generator as my testing set which I use every time I run a model to asses overfitting. Val_generator data is used only once on the final model, to asses how well the model does on data is has never seen before. This clarification is important because for every model that is run, all of the epochs are printed out, and the metrics/results for testing data are referred to as 'val.' Unless otherwise specified, any metric starting with 'val' in the epich print outs is really testing data.

# **Data Understanding**

This data is composed of a series of Brain MRIs consisting of scans which contain a tumor and those that do not. The data actually comes from an existing kaggle dataset (["Brain Tumor Classification (MRI)"](https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri)), which further divided the tumor data into three different types: Glioma, Meningioma, and Pituitary. I downloaded this data and combined all the tumor scans into one category and then uploaded it onto kaggle, so that I could make this a binary classification question. There are a variety of different planes, or perspectives, from which the scans are taken; some are sagital scans (plane that shows the side of the brain), some are coronal scans (plane that shows the back of the brain, at varying depths), and some are transverse (plane that shows the top of the brain, at varying depths; like a bird's eye view). The dataset contains MRIs taken with a variety of different methods, namely T1, T2, and FLAIR. Each of these different methods results in an MRI image with varying levels of brightness and contrast. The data contains 2,764 tumor MRIs and 500 normal MRIs, so the dataset is very imbalanced. Each image uses all three color channels, and each is a different size, so I standardized all images to be 200 x 200 x 3.

## **Data Augmentation**

Because a total of 2,870 files (number of files in the training set) is a pretty small number of images to use for training a neural network, data augmentation is key, since it can help mimic the effect of having a larger number of images. The way I implemented these techiques was by using the ImageDataGenerator from Keras. The features I decided to tweak for augmentation were zoom range, rotation range, brightness range, and horizontal flipping. I decided to provided a range of different zoom values and rotation degrees because how much zoom and the angle of how the brain is positioned in an MRI image can vary a little, and sp producing images with varying levels of zoom and rotation is a realistic way to mimic the effect of having more images. I decided to provide a range for brightness level because as mentioned in the 'Data Understanding' paragraph above, the dataset contains a variety of images with different levels of brightness and contrast, and so producing images with different levels of brightness is a realistic way to mimic the effect of having more images. I decided to flip some images along the horizontal axis, which translates to a left right flip, because regions of the brain are very symmetrical along the left/right axis. I did not include vertical flipping as part of data augmentation, because top/bottom parts of the brain are not symmetrical. I also decided not to shear any images, because shearing stretches and distorts regions of an image, and for brain scans it is very important to preserve the correct anatomical structure of the brain, as discussed in this [this dataset](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6917660/).

## **Taking a Look at a few different individual images**

In [ ]:
tumor1 = PIL.Image.open('../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/AllTumorsTrain/p (274).jpg')
tumor1

In [ ]:
# Figuring out the number of color channels
tumor1.mode
# It is 'RGB', meaning that there are three color channels 471, 490

In [ ]:
images = ['../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/no_tumor_train/image (11).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/no_tumor_train/image(115).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/no_tumor_train/image(108).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/no_tumor_train/image(100).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/no_tumor_train/image(132).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/no_tumor_train/image(145).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/AllTumorsTrain/gg (105).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/AllTumorsTrain/gg (12).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/AllTumorsTrain/m (126).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/AllTumorsTrain/m (130).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/AllTumorsTrain/p (471).jpg',
         '../input/resortedbraintumorclassificationmridata/Brain_MRI_Tumor_Images/Training/AllTumorsTrain/p (490).jpg']

In [ ]:
# from https://stackoverflow.com/questions/48435229/how-to-plot-a-list-of-image-in-loop-using-matplotlib/48435411
fig, axes = plt.subplots(2, 6, figsize=(18, 10))
rows = 2

for num, x in enumerate(images):
    img = PIL.Image.open(x)
    re_img = img.resize((200,200))
    ax = axes[num // 6, num % 6]
    ax.imshow(re_img)
    

fig.suptitle('Normal Images on Top Row, Tumor Images on Bottom Row', fontsize=20)
plt.tight_layout()
plt.show()

## **Taking a look at the class imbalance**

### **First looking at training data imbalance**

In [ ]:
# looking at how the categories are encoded
train_generator.class_indices
# no tuumor data is encoded as zero, tumor data is encoded as one

In [ ]:
# Looking at the place where all labels for training data are stored 
train_generator.classes

In [ ]:
# Making a DataFrame out of the training data labels
train_tumors = pd.DataFrame(train_generator.classes)
train_values = train_tumors.value_counts()
train_values

#### **The ratio of images with tumors to those without is 2327:372, or 6.255:1.**

In [ ]:
# Making subsets of the dataframe for visualization purposes
train_tumors.rename(columns={0:'Tumor/No Tumor'}, inplace=True)
train_no_tumor = len(train_tumors[train_tumors['Tumor/No Tumor'] == 0])
train_tumor = len(train_tumors[train_tumors['Tumor/No Tumor'] == 1])

In [ ]:
# Barplot for visually assessing the training data class imbalance
plt.figure(figsize=(10,8))
sns.set(font_scale=1.4)
sns.barplot(['No Tumor', 'Tumor'], [train_no_tumor, train_tumor])
plt.ylabel("Number of Images")
plt.title('Distribution of Brain MRIs with and without Tumor');


### **Now taking a look at the test data imbalance**

In [ ]:
# Making a DataFrame of testing data labels
test_tumors = pd.DataFrame(test_generator.classes)
test_values = test_tumors.value_counts()
test_values

#### **The ratio of images with tumors to those without is 289:105, or 2.75:1**

In [ ]:
# Making subsets of the dataframe for plotting purposes
test_tumors.rename(columns={0:'Tumor/No Tumor'}, inplace=True)
test_no_tumor = len(test_tumors[test_tumors['Tumor/No Tumor'] == 0])
test_tumor = len(test_tumors[test_tumors['Tumor/No Tumor'] == 1])

In [ ]:
# Barplot for visually assessing the class imbalance in the testing data
plt.figure(figsize=(10,8))
sns.set(font_scale=1.4)
sns.barplot(['No Tumor', 'Tumor'], [test_no_tumor, test_tumor])
plt.ylabel("Number of Images")
plt.title('Distribution of Brain MRIs with and without Tumor in Testing Data');


## **Baseline CNN Model**

In [ ]:
# Building the first baseline model; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
baseline = keras.Sequential()
baseline.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
baseline.add(layers.MaxPooling2D(2,2))
baseline.add(layers.Conv2D(64, (3,3), activation='relu'))
baseline.add(layers.MaxPooling2D(2,2))

baseline.add(layers.Flatten())
baseline.add(layers.Dense(128, activation='relu'))
baseline.add(layers.Dense(1, activation='sigmoid'))

baseline.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [ ]:
# Has been rerun
# Info for how to construct steps_per_epoch value from:
# https://stackoverflow.com/questions/46010565/checking-validation-results-in-keras-shows-only-50-correct-clearly-random
baseline_results = baseline.fit_generator(train_generator,
                                         steps_per_epoch=2699 // 20 +1,# number of samples / batch size plus one, so that every batch is included
                                         epochs=10,
                                        validation_data=test_generator,
                                         validation_steps= 394 // 20+1)

In [ ]:
visualize_training_results(baseline_results)

**Analysis of Model**
 
In the last epoch, the training accuracy is 93% and testing accuracy is 58%, with a training loss of 17% and a testing loss of 76%. Testing recall is 50% and testing precision is 86%. Obviously, the model is overfitting, and loss is quite high; the recall score is very low as well. In the next model iteration, I will add another dense layer, which will hopefully help the model pick up on more patterns, and some dropout layers for a form of regularization.

## **Adding another Dense layer and Dropout layers**

In [ ]:
# Adding another dense layer and a couple of dropout layers; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
layers_drop = keras.Sequential()
layers_drop.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
layers_drop.add(layers.MaxPooling2D(2,2))
layers_drop.add(layers.Conv2D(64, (3,3), activation='relu'))
layers_drop.add(layers.MaxPooling2D(2,2))

layers_drop.add(layers.Flatten())
layers_drop.add(layers.Dense(128, activation='relu'))
layers_drop.add(layers.Dropout(0.3))
layers_drop.add(layers.Dense(64, activation='relu'))
layers_drop.add(layers.Dropout(0.3))
layers_drop.add(layers.Dense(1, activation='sigmoid'))

layers_drop.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [ ]:
# has been rerun
# Fitting the model
layers_drop_results = layers_drop.fit_generator(train_generator,
                                         steps_per_epoch=2699 // 20+1,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator,
                                        validation_steps= 394 // 20+1)

In [ ]:
# Look at how accuracy and loss change across the epochs, for training and testing data
visualize_training_results(layers_drop_results)

**Analysis of Model**

In this iteration, training accuracy ends up at 92%, and testing ends up at 70%, so the model is still overfitting. As for loss, training loss is 21% and testing loss is 57%, which is not drastically different from the last model. Testing recall is 65% and testing precision is 91%. In the next model iteration I am going to account for the class imabalance, and the added layer and dropout layers might perform better in this iteration.

## **Accounting for class imbalance**

In [ ]:
# Accounting for class imbalance; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

class_ld = keras.Sequential()
class_ld.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_ld.add(layers.MaxPooling2D(2,2))
class_ld.add(layers.Conv2D(64, (3,3), activation='relu'))
class_ld.add(layers.MaxPooling2D(2,2))

class_ld.add(layers.Flatten())
class_ld.add(layers.Dense(128, activation='relu'))
class_ld.add(layers.Dropout(0.3))
class_ld.add(layers.Dense(64, activation='relu'))
class_ld.add(layers.Dropout(0.3))
class_ld.add(layers.Dense(1, activation='sigmoid'))

class_ld.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights = {0: 6.255, # NO TUMOR
          1:1} # TUMOR 
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
# has been rerun
# Fitting the model
class_ld_results = class_ld.fit_generator(train_generator,
                                          class_weight=weights,
                                         steps_per_epoch=2699//20+1,# number of samples / batch size +1, so that no batch is left out
                                         epochs=10,
                                         validation_data=test_generator,
                                         validation_steps=394//20+1)

In [ ]:
# Look at how accuracy and loss change across the epochs, for training and testing data
visualize_training_results(class_ld_results)

**Analysis of Model**

In this model iteration, training accuracy was about 87% and testing accuracy is about 49%, so the model is still overfitting, more so than in the last model. Loss for training is at around 58% and testing loss is around 133%. Testing recall is 32% and testing precision is 95%; it is clear that accounting for class imbalance in this way has made the model worse. Further research lead me to a [great article](https://www.analyticsvidhya.com/blog/2020/10/improve-class-imbalance-class-weights/), which discusses how class weights should be calculated. It mentions how adding too much weight to the minority class can sometimes cause the model to show a bias towards it, getting more images in the majority class wrong and making the model less robust. The calculation for what weight to put on the minority class is: total # samples / (2 * number of samples in the minority class). This calculation applied to the training data samples is as follows: 2870/(2*395), which comes out to be about 3.63. So, the class weight for the minority class, in this case "no_tumor" MRIs, should be close to 3.63. In the next model, I will see if a class weight of three applied to the no tumor images will be beneficial.

## **Adjusting Class Weights of Previous Model**

In [ ]:
# Accounting for class imbalance; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

adjclass_ld = keras.Sequential()
adjclass_ld.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
adjclass_ld.add(layers.MaxPooling2D(2,2))
adjclass_ld.add(layers.Conv2D(64, (3,3), activation='relu'))
adjclass_ld.add(layers.MaxPooling2D(2,2))

adjclass_ld.add(layers.Flatten())
adjclass_ld.add(layers.Dense(128, activation='relu'))
adjclass_ld.add(layers.Dropout(0.3))
adjclass_ld.add(layers.Dense(64, activation='relu'))
adjclass_ld.add(layers.Dropout(0.3))
adjclass_ld.add(layers.Dense(1, activation='sigmoid'))

adjclass_ld.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
# Incorporating adjusted class weights; compiling and fitting the model
weights3 = {0: 3, # NO TUMOR
          1:1.} # TUMOR 
# there are 6.255 times as many images of MRIs with tumors than without, but acording to the class weight calculation, I should 
# use a number close to 3.63 for the minority class, in this case the no tumor, or "zero" class.

In [ ]:
# has been rerun
# Fitting the model
adjclass_ld_results = adjclass_ld.fit_generator(train_generator,
                                          class_weight=weights3,
                                         steps_per_epoch=2699//20+1,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator,
                                         validation_steps=394//20+1)

In [ ]:
visualize_training_results(adjclass_ld_results)

**Analysis of model**

Training accuracy is 91% and testing accuracy is 60%, with a training loss of 35% and a testing loss of 80%. Testing recall is 49% and testing precision is 93%. Although this model still requires much fine tuning, it is clear that adjusting the class weights made it perform a bit better, so these weights will be used from now on. In the next iteration, I will try incorporating class weights.

## **Model with Batch Normalization** 

In [ ]:
# Adding Batch Normalization; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
class_n = keras.Sequential()
class_n.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_n.add(layers.BatchNormalization())
class_n.add(layers.MaxPooling2D(2,2))
class_n.add(layers.Conv2D(64, (3,3), activation='relu'))
class_n.add(layers.BatchNormalization())
class_n.add(layers.MaxPooling2D(2,2))
class_n.add(layers.Conv2D(128, (3,3), activation='relu'))
class_n.add(layers.BatchNormalization())
class_n.add(layers.MaxPooling2D(2,2))

class_n.add(layers.Flatten())
class_n.add(layers.Dense(128, activation='relu'))
class_n.add(layers.Dropout(0.3))
class_n.add(layers.Dense(1, activation='sigmoid'))

class_n.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights3 = {0: 3, # NO TUMOR
          1:1} # TUMOR 
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
early_stop2 = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [ ]:
class_n_results = class_n.fit_generator(train_generator,
                                          class_weight=weights3,
                                         steps_per_epoch=2699//20+1,# number of samples / batch size
                                         epochs=20,
                                         callbacks=early_stop2,
                                         validation_data=test_generator,
                                         validation_steps=394//20+1)

In [ ]:
# Look at how accuracy and loss change across the epochs, for training and testing data
visualize_training_results(class_n_results)

**Analysis of Model**

 In the epoch with the lowest testing loss, training accuracy is 91% and testing accuracy is 66%, with a training loss of 30% and a testing loss of 56%. Testing recall is 57% and testing precision is 96%. Loss has been decreased in this model iteration compared to the last.
 
 Because batch normalization makes the network more stable, it is possible to use larger learning rates, which could potentially help the model reach optimal accuracy and minimal loss more quickly, so that is what I will try next.

## **Using a Bigger Learning Rate since I am using Batch Normalization**

In [ ]:
# Establishing an instance of Adam with a bigger learning rate
adam_mlr = keras.optimizers.Adam(epsilon=0.01)

In [ ]:
# Using Batch Normalization; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
class_na = keras.Sequential()
class_na.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_na.add(layers.BatchNormalization())
class_na.add(layers.MaxPooling2D(2,2))
class_na.add(layers.Conv2D(64, (3,3), activation='relu'))
class_na.add(layers.BatchNormalization())
class_na.add(layers.MaxPooling2D(2,2))
class_na.add(layers.Conv2D(128, (3,3), activation='relu'))
class_na.add(layers.BatchNormalization())
class_na.add(layers.MaxPooling2D(2,2))

class_na.add(layers.Flatten())
class_na.add(layers.Dense(128, activation='relu'))
class_na.add(layers.Dropout(0.3))
class_na.add(layers.Dense(1, activation='sigmoid'))

class_na.compile(loss='binary_crossentropy',
                optimizer=adam_mlr,
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weights3 = {0: 3, # NO TUMOR
          1:1} # TUMOR 
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
# Using a bigger learning rate; fitting the model
class_na_results = class_na.fit_generator(train_generator,
                                          class_weight=weights3,
                                         steps_per_epoch=2699//20+1,# number of samples / batch size
                                         epochs=20,
                                         callbacks=early_stop2,
                                         validation_data=test_generator,
                                         validation_steps=394//20+1)

In [ ]:
# Look at how accuracy and loss change across the epochs, for training and testing data
visualize_training_results(class_na_results)

**Analysis of Model**

In the epoch with the lowest testing loss, training accuracy is 91% while testing accuracy is 75%. Training loss is 27%, while testing loss is 43%. 
Testing recall is 89% and testing precision is 90%. Testing accuracy is a little greater and testing loss is a bit lower than in the last iteration, so it looks like using a bigger learning rate in combination with Batch Normalization was a good idea. 

## **Model with Batch Normalization and more Dropout Layers**

In [ ]:
# Accounting for the class imbalance; structure is modified from one shown on:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
class_nd = keras.Sequential()

class_nd.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
class_nd.add(layers.BatchNormalization())
class_nd.add(layers.MaxPooling2D(2,2))
class_nd.add(layers.Dropout(0.25))

class_nd.add(layers.Conv2D(64, (3,3), activation='relu'))
class_nd.add(layers.BatchNormalization())
class_nd.add(layers.MaxPooling2D(2,2))
class_nd.add(layers.Dropout(0.25))

class_nd.add(layers.Conv2D(128, (3,3), activation='relu'))
class_nd.add(layers.BatchNormalization())
class_nd.add(layers.MaxPooling2D(2,2))
class_nd.add(layers.Dropout(0.25))

class_nd.add(layers.Flatten())
class_nd.add(layers.Dense(128, activation='relu'))
class_nd.add(layers.Dropout(0.4))
class_nd.add(layers.Dense(1, activation='sigmoid'))

class_nd.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
weight3 = {0: 3, # NO TUMOR
          1:1} # TUMOR 
# there are 6.255 times as many images of MRIs with tumors than without

In [ ]:
# Fitting the model
class_nd_results = class_nd.fit_generator(train_generator,
                                          class_weight=weights3,
                                         steps_per_epoch=2699//20+1,# number of samples / batch size
                                         epochs=20,
                                          callbacks=early_stop2,
                                         validation_data=test_generator,
                                         validation_steps=394//20+1)

In [ ]:
# Look at how accuracy and loss change across the epochs, for training and testing data
visualize_training_results(class_nd_results)

**Analysis of Model**

Training accuracy of the epoch with the lowest loss is 86% while testing accuracy is 74%. Training loss is 41% while testing loss is 68%. Testing recall is 100% and testing precision is 74%. Training and Testing loss have both increased, so the dropout layers added after each max pooling step may not be particularly beneficial to the model. Next, I will try using pretrained neural networks.

## **Using the Pre-Trained VGG-19 Weights (this is my FSM)**

In [ ]:
from keras.applications.vgg19 import VGG19
cnn_vgg = VGG19(weights='imagenet',
               include_top=False,
               input_shape=(200,200,3))

In [ ]:
cnn_vgg.summary()


In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
pretrained = keras.Sequential()
pretrained.add(cnn_vgg)
pretrained.add(layers.Flatten())
pretrained.add(layers.Dense(128, activation='relu'))
pretrained.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
def Freeze_Pretrained_Base(pretrain, network):
    pretrain.trainable = False
    for layer in network.layers:
        print(layer.name, layer.trainable)
    print(len(network.trainable_weights))

In [ ]:
Freeze_Pretrained_Base(cnn_vgg, pretrained)

In [ ]:

pretrained.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

pretrained_results = pretrained.fit_generator(train_generator,
                                         steps_per_epoch=2699//20+1,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator,
                                             validation_steps=394//20+1)

In [ ]:
# Look at how accuracy and loss change across the epochs, for training and testing data
visualize_training_results(pretrained_results)

**Analysis of Model**

The epoch with the lowest testing loss has a training accuracy of 95% and a testing accuracy of 82%, with a training loss of 14% and a testing loss of 45%. Testing recall is 87%, and testing precision is 88%. Testing loss is lower and testing accuracy is higher than in the previous model, so using pretrianed models looks promising. In the next iteration, I will try fine tuning this network by adding dropout layers and unfreezing some of the outer layers of the pretrained network so they can learn from the images in this dataset.

## **Unfreezing Layers in the Pretrained VGG-19 Network**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
b5c1c2 = keras.Sequential()
b5c1c2.add(cnn_vgg)
b5c1c2.add(layers.Flatten())
b5c1c2.add(layers.Dense(128, activation='relu'))
b5c1c2.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
# List for unfreezing of layers function
unfreeze = ['block5_conv1', 'block5_conv2']

In [ ]:
# Unfreezing some of outer layers of VGG19 pretrained network
Unfreeze_Layers(cnn_vgg, unfreeze)

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
def Unfreeze_Layers(pretrain, layer_list):
    pretrain.trainable = True
    for layer in  pretrain.layers:
        if layer.name in layer_list:
            layer.trainable = True
        else:
            layer.trainable = False
        
    for layer in pretrain.layers:
        print(layer.name, layer.trainable)
    print(len(pretrain.trainable_weights))

In [ ]:
# Compiling and Fitting the model
b5c1c2.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

b5c1c2_results = b5c1c2.fit_generator(train_generator,
                                         steps_per_epoch=2699//20+1,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator,
                                         validation_steps=394//20+1)

In [ ]:
# Compiling and Fitting the model
b5c1c2.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

b5c1c2_results = b5c1c2.fit_generator(train_generator,
                                         steps_per_epoch=2699//20+1,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator,
                                         validation_steps=394//20+1)

**Analysis of Model**

In the epoch with the lowest epoch, training accuracy is 99% and testing accuracy is 91%, with a training loss of 5% and a testing loss of 27%. Testing recall is 89% and testing precision is 98%. Testing accuracy and loss have increased considerably since the last iteration! Unfreezing some of the outer layers of the pretrained network seems to have helped the model better learn patterns in the MRI images.